# Manually converting calibrated data to superstar

In [2]:
import dotenv

# load in environment variables
dotenv.load_dotenv()

import datetime as dt
import magicpy.analysis as analysis
import magicpy.datalevel as dl
import magicpy.periods as periods
import magicpy.moons as moons
import magicpy.zeniths as zeniths
import magicpy.mars as mars

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=50, progress_bar=True)
import uproot
import numpy as np
from pathlib import Path
import pandas as pd

import sys
import os
import subprocess

INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
mc = analysis.MC(
    period=periods.ST0307,
    zenith=zeniths.za05to35,
)

Loading existing analysis from /remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0307/MC_ST0307_za05to35_None.magicpy
INFO: RFs not found for: ST0307 za05to35 weak moon.
INFO: RFs not found for: ST0307 za05to35 moderate moon.
INFO: RFs not found for: ST0307 za05to35 decent moon.
INFO: RFs not found for: ST0307 za05to35 strong moon.
INFO: RFs not found for: ST0307 za05to35 high moon.
INFO: RFs not found for: ST0307 za05to35 super moon.
INFO: RFs not found for: ST0307 za05to35 reduced_decent moon.
INFO: RFs not found for: ST0307 za05to35 reduced_strong moon.
INFO: RFs not found for: ST0307 za05to35 reduced_high moon.


In [3]:
# point to MCs on scratch for faster running

old_path = "/remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0307/za05to35/weak"
new_path = "/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak"
# new_path = "/remote/lstdata01/jgreen/MAGIC_MC/root/gammas"

# replace the old path with the new path in mc.runs.local_path dataframe
mc._runs["local_path_mnt"] = mc._runs.local_path.apply(
    lambda x: Path(str(x).replace(old_path, new_path)).absolute()
)

# separate M1 and M2 into different dataframes

calib_m1 = mc._runs[mc._runs.data_level == dl.CalibratedM1].sort_values(by="run_number")
calib_m2 = mc._runs[mc._runs.data_level == dl.CalibratedM2].sort_values(by="run_number")

# get some test files
test_run_m1 = calib_m1.iloc[0]
test_run_m2 = calib_m2.iloc[0]

## Run one

In [8]:
def process_calibrated(
    run_number: int,
    calibrated_m1_file: Path | str,
    calibrated_m2_file: Path | str,
    star_output_dir: Path | str,
    superstar_output_dir: Path | str | None = None,
    to_parquet: bool = True,
    parquet_file_prefix: str | None = None,
    overwrite_root: bool = True,
):

    # load in environment variables
    env = dict(dotenv.dotenv_values())
    
    if not env.get("MARSSYS"):
        raise ValueError("MARSSYS environment variable not set")
    if not env.get("ROOTSYS"):
        raise ValueError("ROOTSYS environment variable not set")
    
    if not superstar_output_dir:
        superstar_output_dir = star_output_dir
        
    if not star_output_dir.exists():
        star_output_dir.mkdir(parents=True)
    if not superstar_output_dir.exists():
        superstar_output_dir.mkdir(parents=True)
    
    if not isinstance(calibrated_m1_file, Path):
        calibrated_m1_file = Path(calibrated_m1_file)
    if not isinstance(calibrated_m2_file, Path):
        calibrated_m2_file = Path(calibrated_m2_file)

    # ensure the files exist
    if not calibrated_m1_file.exists():
        raise FileNotFoundError(f"Calibrated M1 file not found: {calibrated_m1_file}")
    if not calibrated_m2_file.exists():
        raise FileNotFoundError(f"Calibrated M2 file not found: {calibrated_m2_file}")
    
    starm1 = mars.Star(
        input_files=[test_run_m1.local_path_mnt],
        analysis_path=star_output_dir,
        output_dir=star_output_dir,
        nickname=f"StarM1-DHBW-gamma-{run_number}",
        telescope_number=1,
        is_mc=True,
    )

    starm2 = mars.Star(
        input_files=[test_run_m2.local_path_mnt],
        analysis_path=star_output_dir,
        output_dir=star_output_dir,
        nickname=f"StarM2-DHBW-gamma-{run_number}",
        telescope_number=2,
        is_mc=True,
    )
    
    resm1 = starm1.run(
        file=test_run_m1.local_path_mnt,
        save_all=True,
        overwrite=overwrite_root,
    )

    resm2 = starm2.run(
        file=test_run_m2.local_path_mnt,
        save_all=True,
        overwrite=overwrite_root,
    )
    
    if resm1.get("return_code") != 0:
        raise ValueError(f"Error running M1 star: \n{resm1['error']}")
    if resm2.get("return_code") != 0:
        raise ValueError(f"Error running M2 star: \n{resm2['error']}")
    
    star_m1_filepath = resm1["star_file_path"]
    star_m2_filepath = resm2["star_file_path"]
    
    star_m1_regex = star_m1_filepath.parent / f"{star_m1_filepath.stem}*.root"
    star_m2_regex = star_m2_filepath.parent / f"{star_m2_filepath.stem}*.root"

    superstar = mars.SuperStar(
        input_files=[star_m1_regex, star_m2_regex],
        analysis_path=superstar_output_dir,
        output_dir=superstar_output_dir,
        nickname=f"SuperStar-DHBW-gamma-{test_run_m1.run_number}",
        is_mc=True,
    )
    
    ssres = superstar.run(
        m1_file=star_m1_regex,
        m2_file=star_m2_regex,
        is_mc=True,
        log_file=superstar_output_dir / f"SuperStar-DHBW-gamma-{run_number}.log",
        overwrite=overwrite_root,
    )
    
    if ssres.get("return_code") != 0:
        raise ValueError(f"Error running SuperStar: \n{ssres['error']}")
    
    superstar_filepath = ssres["superstar_file"]
    
    # run the writeImages2uproot macro
    eof = f'root -b -l -q {env["MARSSYS"]}/macros/writeImages2uproot.C("{superstar_filepath.name}")'
    
    process = subprocess.Popen(
        eof.split(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        cwd=superstar_output_dir,
        env=superstar.env | env,
    )
    
    output, error = process.communicate()
    
    if process.returncode != 0:
        raise ValueError(f"Error running writeImages2uproot.C: {error}")
    
    res = {}
    params = [
        "Events;2/MMcEvt_1./MMcEvt_1.fEvtNumber",
        "Events;2/MMcEvt_1./MMcEvt_1.fEnergy",
        "Events;2/MMcEvt_1./MMcEvt_1.fTheta",
        "Events;2/MMcEvt_1./MMcEvt_1.fPhi",
        "Events;2/MMcEvt_1./MMcEvt_1.fTelescopeTheta",
        "Events;2/MMcEvt_1./MMcEvt_1.fTelescopePhi",
        "Events;2/MMcEvt_1./MMcEvt_1.fZFirstInteraction",
        "Events;2/MMcEvt_2./MMcEvt_2.fEvtNumber",
        # "Events;2/MMcEvt_2./MMcEvt_2.fEnergy",
        # "Events;2/MMcEvt_2./MMcEvt_2.fTheta",
        # "Events;2/MMcEvt_2./MMcEvt_2.fPhi",
        # "Events;2/MMcEvt_2./MMcEvt_2.fTelescopeTheta",
        # "Events;2/MMcEvt_2./MMcEvt_2.fTelescopePhi",
        # "Events;2/MMcEvt_2./MMcEvt_2.fZFirstInteraction",
        "Events;2/MHillas_1./MHillas_1.fLength",
        "Events;2/MHillas_1./MHillas_1.fWidth",
        "Events;2/MHillas_1./MHillas_1.fDelta",
        "Events;2/MHillas_1./MHillas_1.fSize",
        "Events;2/MHillas_1./MHillas_1.fMeanX",
        "Events;2/MHillas_1./MHillas_1.fMeanY",
        "Events;2/MHillas_1./MHillas_1.fSinDelta",
        "Events;2/MHillas_1./MHillas_1.fCosDelta",
        "Events;2/MHillas_2./MHillas_2.fLength",
        "Events;2/MHillas_2./MHillas_2.fWidth",
        "Events;2/MHillas_2./MHillas_2.fDelta",
        "Events;2/MHillas_2./MHillas_2.fSize",
        "Events;2/MHillas_2./MHillas_2.fMeanX",
        "Events;2/MHillas_2./MHillas_2.fMeanY",
        "Events;2/MHillas_2./MHillas_2.fSinDelta",
        "Events;2/MHillas_2./MHillas_2.fCosDelta",
        "Events;2/MStereoPar./MStereoPar.fDirectionX",
        "Events;2/MStereoPar./MStereoPar.fDirectionY",
        "Events;2/MStereoPar./MStereoPar.fDirectionZd",
        "Events;2/MStereoPar./MStereoPar.fDirectionAz",
        "Events;2/MStereoPar./MStereoPar.fDirectionDec",
        "Events;2/MStereoPar./MStereoPar.fDirectionRA",
        "Events;2/MStereoPar./MStereoPar.fTheta2",
        "Events;2/MStereoPar./MStereoPar.fCoreX",
        "Events;2/MStereoPar./MStereoPar.fCoreY",
        "Events;2/MStereoPar./MStereoPar.fM1Impact",
        "Events;2/MStereoPar./MStereoPar.fM2Impact",
        "Events;2/MStereoPar./MStereoPar.fM1ImpactAz",
        "Events;2/MStereoPar./MStereoPar.fM2ImpactAz",
        "Events;2/MStereoPar./MStereoPar.fMaxHeight",
        "Events;2/MStereoPar./MStereoPar.fXMax",
        "Events;2/MStereoPar./MStereoPar.fCherenkovRadius",
        "Events;2/MStereoPar./MStereoPar.fCherenkovDensity",
        # "Events;2/MStereoPar./MStereoPar.fEnergy",
        # "Events;2/MStereoPar./MStereoPar.fEnergyUncertainty",
        # "Events;2/MStereoPar./MStereoPar.fEnergyDiscrepancy",
        "Events;2/MStereoPar./MStereoPar.fPhiBaseLineM1",
        "Events;2/MStereoPar./MStereoPar.fPhiBaseLineM2",
        "Events;2/MStereoPar./MStereoPar.fImageAngle",
        # "Events;2/MStereoPar./MStereoPar.fDispRMS",
        # "Events;2/MStereoPar./MStereoPar.fDispDiff2",
        "Events;2/MStereoPar./MStereoPar.fCosBSangle",
        "Events;2/MPointingPos_1./MPointingPos_1.fZd",
        # "Events;2/MPointingPos_2./MPointingPos_2.fZd",
        "Events;2/MPointingPos_1./MPointingPos_1.fAz",
        #"Events;2/MPointingPos_2./MPointingPos_2.fAz",
        "Events;2/MHillasTimeFit_1./MHillasTimeFit_1.fP1Grad",
        "Events;2/MHillasTimeFit_2./MHillasTimeFit_2.fP1Grad",
        "Events;2/MMcEvt_1./MMcEvt_1.fImpact",
        "Events;2/MMcEvt_2./MMcEvt_2.fImpact",
        "Events;2/MHillasSrc_1./MHillasSrc_1.fAlpha",
        "Events;2/MHillasSrc_1./MHillasSrc_1.fDist",
        "Events;2/MHillasSrc_1./MHillasSrc_1.fCosDeltaAlpha",
        "Events;2/MHillasSrc_1./MHillasSrc_1.fDCA",
        "Events;2/MHillasSrc_1./MHillasSrc_1.fDCADelta",
        "Events;2/MHillasSrc_2./MHillasSrc_2.fAlpha",
        "Events;2/MHillasSrc_2./MHillasSrc_2.fDist",
        "Events;2/MHillasSrc_2./MHillasSrc_2.fCosDeltaAlpha",
        "Events;2/MHillasSrc_2./MHillasSrc_2.fDCA",
        "Events;2/MHillasSrc_2./MHillasSrc_2.fDCADelta",
    ]

    with uproot.open(superstar_filepath) as f:
        # get the event number
        res["event_number"] = f["Events;2/MMcEvt_1./MMcEvt_1.fEvtNumber"].array(
            library="np"
        )
        res["run_number"] = np.full_like(res["event_number"], run_number)

        for k in params:
            title = k.split("/")[-1]
            res[title] = f[k].array().to_numpy()

        res["image_m1"] = [
            i.tojson() for i in f["Events;2/UprootImageOrig_1"].array(library="np")
        ]
        res["image_m2"] = [
            i.tojson() for i in f["Events;2/UprootImageOrig_2"].array(library="np")
        ]
        res["clean_image_m1"] = [
            i.tojson() for i in f["Events;2/UprootImageOrigClean_1"].array(library="np")
        ]
        res["clean_image_m2"] = [
            i.tojson() for i in f["Events;2/UprootImageOrigClean_2"].array(library="np")
        ]


    # also extract images and timing information from the calibrated files
    calibrated_res = {
        1: {},
        2: {},
    }
    for file, telescope in zip(
        [calibrated_m1_file, calibrated_m2_file], [1, 2]
    ):
        with uproot.open(file) as f:
            event_numbers = f["Events;1/MMcEvt./MMcEvt.fEvtNumber"].array(library="np")
            calibrated_res[telescope][f"calibrated_event_number_M{telescope}"] = (
                event_numbers.tolist()
            )
            n_events = len(event_numbers)

            # don't need to extract the images again
            # calibrated_res[telescope][f"calibrated_images_M{telescope}"] = (
            #     f["Events;1/MCerPhotEvt./MCerPhotEvt.fPixels/MCerPhotEvt.fPixels.fPhot"]
            #     .array(library="np")
            #     .tolist()
            # )

            arrival_times = []
            timing_data = f["Events;1/MArrivalTime./MArrivalTime.fData"].array()
            for t in timing_data:
                d = t.to_list()
                arrival_times.append(d)

            calibrated_res[telescope][f"calibrated_timing_M{telescope}"] = (
                np.array(arrival_times).reshape(n_events, len(d)).tolist()
            )

            # reformat calibrated res into {event_number: {m1_image: value, m2_image: value, ...}}
            event_key = f"calibrated_event_number_M{telescope}"
            calibrated_res[telescope] = {
                event_number: {k: v[i] for k, v in calibrated_res[telescope].items()}
                for i, event_number in enumerate(calibrated_res[telescope][event_key])
            }

            # remove event numbers over 10000
            calibrated_res[telescope] = {
                k: v for k, v in calibrated_res[telescope].items() if k < 10000
            }


    # merge based on event_number
    calibrated_merged = pd.DataFrame.from_dict(calibrated_res[1], orient="index").merge(
        pd.DataFrame.from_dict(calibrated_res[2], orient="index"),
        left_index=True,
        right_index=True,
    )

    # merge with the superstar data
    superstar_df = pd.DataFrame.from_dict(res)
    merged = superstar_df.merge(
        calibrated_merged, left_on="event_number", right_index=True
    )

    drop = [
        "MMcEvt_2.fEvtNumber",
        "MMcEvt_1.fEvtNumber",
        "calibrated_event_number_M1",
        "calibrated_event_number_M2",
    ]

    merged = merged.drop(columns=drop)

    rename_dict = {
        # Event Info
        "event_number": "event_number",
        "run_number": "run_number",
        
        # Monte Carlo Truth (both)
        "MMcEvt_1.fEnergy": "true_energy",
        "MMcEvt_1.fTheta": "true_theta",
        "MMcEvt_1.fPhi": "true_phi",
        "MMcEvt_1.fTelescopeTheta": "true_telescope_theta",
        "MMcEvt_1.fTelescopePhi": "true_telescope_phi",
        "MMcEvt_1.fZFirstInteraction": "true_first_interaction_height",
        
        # Monte Carlo Truth (M2)
        "MMcEvt_1.fImpact": "true_impact_m1",
        
        # Monte Carlo Truth (M2)
        "MMcEvt_2.fImpact": "true_impact_m2",
        
        # Hillas Parameters (M1)
        "MHillas_1.fLength": "hillas_length_m1",
        "MHillas_1.fWidth": "hillas_width_m1",
        "MHillas_1.fDelta": "hillas_delta_m1",
        "MHillas_1.fSize": "hillas_size_m1",
        "MHillas_1.fMeanX": "hillas_cog_x_m1",
        "MHillas_1.fMeanY": "hillas_cog_y_m1",
        "MHillas_1.fSinDelta": "hillas_sin_delta_m1",
        "MHillas_1.fCosDelta": "hillas_cos_delta_m1",
        
        # Hillas Parameters (M2)
        "MHillas_2.fLength": "hillas_length_m2",
        "MHillas_2.fWidth": "hillas_width_m2",
        "MHillas_2.fDelta": "hillas_delta_m2",
        "MHillas_2.fSize": "hillas_size_m2",
        "MHillas_2.fMeanX": "hillas_cog_x_m2",
        "MHillas_2.fMeanY": "hillas_cog_y_m2",
        "MHillas_2.fSinDelta": "hillas_sin_delta_m2",
        "MHillas_2.fCosDelta": "hillas_cos_delta_m2",
        
        # Stereo Parameters
        "MStereoPar.fDirectionX": "stereo_direction_x",
        "MStereoPar.fDirectionY": "stereo_direction_y",
        "MStereoPar.fDirectionZd": "stereo_zenith",
        "MStereoPar.fDirectionAz": "stereo_azimuth",
        "MStereoPar.fDirectionDec": "stereo_dec",
        "MStereoPar.fDirectionRA": "stereo_ra",
        "MStereoPar.fTheta2": "stereo_theta2",
        "MStereoPar.fCoreX": "stereo_core_x",
        "MStereoPar.fCoreY": "stereo_core_y",
        "MStereoPar.fM1Impact": "stereo_impact_m1",
        "MStereoPar.fM2Impact": "stereo_impact_m2",
        "MStereoPar.fM1ImpactAz": "stereo_impact_azimuth_m1",
        "MStereoPar.fM2ImpactAz": "stereo_impact_azimuth_m2",
        "MStereoPar.fMaxHeight": "stereo_shower_max_height",
        "MStereoPar.fXMax": "stereo_xmax",
        "MStereoPar.fCherenkovRadius": "stereo_cherenkov_radius",
        "MStereoPar.fCherenkovDensity": "stereo_cherenkov_density",
        "MStereoPar.fPhiBaseLineM1": "stereo_baseline_phi_m1",
        "MStereoPar.fPhiBaseLineM2": "stereo_baseline_phi_m2",
        "MStereoPar.fImageAngle": "stereo_image_angle",
        "MStereoPar.fCosBSangle": "stereo_cos_between_shower",
        
        # Pointing
        "MPointingPos_1.fZd": "pointing_zenith",
        "MPointingPos_1.fAz": "pointing_azimuth",
        
        # Time Fit
        "MHillasTimeFit_1.fP1Grad": "time_gradient_m1",
        "MHillasTimeFit_2.fP1Grad": "time_gradient_m2",
        
        # Source Parameters (M1)
        "MHillasSrc_1.fAlpha": "source_alpha_m1",
        "MHillasSrc_1.fDist": "source_dist_m1",
        "MHillasSrc_1.fCosDeltaAlpha": "source_cos_delta_alpha_m1",
        "MHillasSrc_1.fDCA": "source_dca_m1",
        "MHillasSrc_1.fDCADelta": "source_dca_delta_m1",
        
        # Source Parameters (M2)
        "MHillasSrc_2.fAlpha": "source_alpha_m2",
        "MHillasSrc_2.fDist": "source_dist_m2",
        "MHillasSrc_2.fCosDeltaAlpha": "source_cos_delta_alpha_m2",
        "MHillasSrc_2.fDCA": "source_dca_m2",
        "MHillasSrc_2.fDCADelta": "source_dca_delta_m2",
        
        # Images and Timing
        "image_m1": "image_m1",
        "image_m2": "image_m2",
        "clean_image_m1": "clean_image_m1",
        "clean_image_m2": "clean_image_m2",
        "calibrated_timing_M1": "timing_m1",
        "calibrated_timing_M2": "timing_m2"
    }

    # Apply the renaming
    merged = merged.rename(columns=rename_dict)

    if to_parquet:

        if not parquet_file_prefix:
            parquet_file_prefix = f"SuperStar_{run_number}"

        output_filename = superstar_output_dir / f"{parquet_file_prefix}.parquet"
        merged.to_parquet(output_filename)
        return output_filename
    
    return merged

In [16]:
output_dir = Path(
    "/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-function"
)

# output_dir = Path("/remote/lstdata01/jgreen/MAGIC_MC/root/gammas/SuperStar-function")

res = process_calibrated(
    run_number=test_run_m1.run_number,
    calibrated_m1_file=test_run_m1.local_path_mnt,
    calibrated_m2_file=test_run_m2.local_path_mnt,
    star_output_dir=output_dir,
    to_parquet=True,
)

In [17]:
df = pd.read_parquet(res)

## run in parallel

In [5]:
# Create separate DataFrames for M1 and M2
m1_df = pd.DataFrame({
    'run_number': calib_m1.run_number,
    'm1_calib_file': calib_m1.local_path_mnt
})

m2_df = pd.DataFrame({
    'run_number': calib_m2.run_number,
    'm2_calib_file': calib_m2.local_path_mnt
})

# Merge the DataFrames on run_number
calib_df = pd.merge(m1_df, m2_df, on='run_number', how='inner')

# Verify we have data
print(f"Found {len(calib_df)} matching M1/M2 file pairs")

Found 4999 matching M1/M2 file pairs


In [9]:
import shutil

output_dir = Path(
    "/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-function"
)

def process_row(row, output_dir):
    """Process a single row from calib_df with self-contained environment setup"""
    # Create a unique output directory for this run number
    process_output_dir = output_dir / f"SuperStar-{row.run_number}"
    process_output_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        # Run the processing
        result_file = process_calibrated(
            run_number=row.run_number,
            calibrated_m1_file=row.m1_calib_file,
            calibrated_m2_file=row.m2_calib_file,
            star_output_dir=process_output_dir,
            to_parquet=True
        )
        
        # Move the result file to the main output directory
        final_path = output_dir / result_file.name
        shutil.move(str(result_file), str(final_path))
        
        # Clean up the temporary directory
        shutil.rmtree(process_output_dir)
        
        return final_path
        
    except Exception as e:
        # Clean up on error
        if process_output_dir.exists():
            shutil.rmtree(process_output_dir)
        raise e

# Update the parallel processing call to include output_dir
results = calib_df.parallel_apply(lambda row: process_row(row, output_dir), axis=1)

# Print summary
print(f"Processed {len(results)} file pairs")
print(f"Output files saved to: {output_dir}")

Processed 4999 file pairs
Output files saved to: /mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-function


In [4]:
df = pd.read_parquet("/mnt/scratch/jgreen/MAGIC_MC/DHBW-parquet/SuperStar-gammas.parquet")

In [12]:
df.to_parquet(output_dir.parent / "SuperStar-gammas.parquet")

## Manually testing 

In [9]:
star_output_dir = Path(
    "/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual"
).absolute()

In [56]:
starm1 = mars.Star(
    input_files=[test_run_m1.local_path_mnt],
    analysis_path=star_output_dir,
    output_dir=star_output_dir,
    nickname=f"StarM1-DHBW-gamma-{test_run_m1.run_number}",
    telescope_number=1,
    is_mc=True,
    env=dict(dotenv.dotenv_values()),
)

starm2 = mars.Star(
    input_files=[test_run_m2.local_path_mnt],
    analysis_path=star_output_dir,
    output_dir=star_output_dir,
    nickname=f"StarM2-DHBW-gamma-{test_run_m2.run_number}",
    telescope_number=2,
    is_mc=True,
    env=dict(dotenv.dotenv_values()),
)

In [57]:
resm1 = starm1.run(
    file=test_run_m1.local_path_mnt,
    save_all=True,
    overwrite=True,
)

resm2 = starm2.run(
    file=test_run_m2.local_path_mnt,
    save_all=True,
    overwrite=True,
)


MARSSYS: /home/iwsatlas1/jgreen/mars/gitlab/Mars ROOTSYS: /home/iwsatlas1/jgreen/micromamba/envs/root6
Running command:
/home/iwsatlas1/jgreen/mars/gitlab/Mars/star -b -f -q -mc --config=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual/runcards/StarM1-DHBW-gamma-821319.rc --ind=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Calibrated_M1/GA_M1_za05to35_8_821319_Y_w0.root --out=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual --log=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual/GA_M1_za05to35_8_821319_Y_w0.log -saveimages -saveimagesclean -savecerevt
MARSSYS: /home/iwsatlas1/jgreen/mars/gitlab/Mars ROOTSYS: /home/iwsatlas1/jgreen/micromamba/envs/root6
Running command:
/home/iwsatlas1/jgreen/mars/gitlab/Mars/star -b -f -q -mc --config=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual/runcards/StarM2-DHBW-gamma-821319.rc --ind=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/

In [29]:
starM1 = Path(
    "/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual/GA_M1_za05to35_8_821319_I_w0.root"
)
starM2 = Path(
    "/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual/GA_M2_za05to35_8_821319_I_w0.root"
)

In [130]:
resm1["star_file_path"].parent / / f"{starM1.stem}*.root"

PosixPath('/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual')

In [30]:
output_dir = Path(
    "/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-manual"
).absolute()

m1_file_regex = starM1.parent / f"{starM1.stem}*.root"
m2_file_regex = starM2.parent / f"{starM2.stem}*.root"

superstar = mars.SuperStar(
    input_files=[m1_file_regex, m2_file_regex],
    analysis_path=output_dir,
    output_dir=output_dir,
    nickname=f"SuperStar-DHBW-gamma-{test_run_m1.run_number}",
    is_mc=True,
)

In [33]:
ssres = superstar.run(
    m1_file=m1_file_regex,
    m2_file=m2_file_regex,
    is_mc=True,
    log_file=output_dir / f"SuperStar-DHBW-gamma-{test_run_m1.run_number}.log",
    overwrite=True,
)

MARSSYS: /home/iwsatlas1/jgreen/mars/gitlab/Mars ROOTSYS: /home/iwsatlas1/jgreen/micromamba/envs/root6
Running command:
/home/iwsatlas1/jgreen/mars/gitlab/Mars/superstar -b -f -q -mc --config=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-manual/runcards/SuperStar-DHBW-gamma-821319.rc --ind1=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual/GA_M1_za05to35_8_821319_I_w0*.root --ind2=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/Star-manual/GA_M2_za05to35_8_821319_I_w0*.root --out=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-manual --log=/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-manual/SuperStar-DHBW-gamma-821319.log


In [32]:
ssres

{'return_code': 127,
 'superstar_file': PosixPath('/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-manual/GA_za05to35_8_821319_S_w0.root'),
 'log_file': PosixPath('/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-manual/SuperStar-DHBW-gamma-821319.log'),
 'runcard': PosixPath('/mnt/scratch/jgreen/MAGIC_MC/root/gammas/ST0307/za05to35/weak/SuperStar-manual/runcards/SuperStar-DHBW-gamma-821319.rc'),
 'output': '',
 'error': '/home/iwsatlas1/jgreen/mars/gitlab/Mars/superstar: error while loading shared libraries: libmars.so: cannot open shared object file: No such file or directory\n'}

## Manually convert

In [ ]:
superstar_filepath = "/remote/lstdata01/jgreen/MAGIC_MC/root/gammas/SuperStar-function/GA_za05to35_8_821319_S_w0.root"
calibrated_m1_file = "/remote/lstdata01/jgreen/MAGIC_MC/root/gammas/SuperStar-function/GA_M1_za05to35_8_821319_I_w0.root"
calibrated_m2_file = "/remote/lstdata01/jgreen/MAGIC_MC/root/gammas/SuperStar-function/GA_M2_za05to35_8_821319_I_w0.root"
run_number = test_run_m1.run_number


res = {}
params = [
    "Events;2/MMcEvt_1./MMcEvt_1.fEvtNumber",
    "Events;2/MMcEvt_1./MMcEvt_1.fEnergy",
    "Events;2/MMcEvt_1./MMcEvt_1.fTheta",
    "Events;2/MMcEvt_1./MMcEvt_1.fPhi",
    "Events;2/MMcEvt_1./MMcEvt_1.fTelescopeTheta",
    "Events;2/MMcEvt_1./MMcEvt_1.fTelescopePhi",
    "Events;2/MMcEvt_1./MMcEvt_1.fZFirstInteraction",
    "Events;2/MMcEvt_2./MMcEvt_2.fEvtNumber",
    "Events;2/MMcEvt_2./MMcEvt_2.fEnergy",
    "Events;2/MMcEvt_2./MMcEvt_2.fTheta",
    "Events;2/MMcEvt_2./MMcEvt_2.fPhi",
    "Events;2/MMcEvt_2./MMcEvt_2.fTelescopeTheta",
    "Events;2/MMcEvt_2./MMcEvt_2.fTelescopePhi",
    "Events;2/MMcEvt_2./MMcEvt_2.fZFirstInteraction",
    "Events;2/MHillas_1./MHillas_1.fLength",
    "Events;2/MHillas_1./MHillas_1.fWidth",
    "Events;2/MHillas_1./MHillas_1.fDelta",
    "Events;2/MHillas_1./MHillas_1.fSize",
    "Events;2/MHillas_1./MHillas_1.fMeanX",
    "Events;2/MHillas_1./MHillas_1.fMeanY",
    "Events;2/MHillas_1./MHillas_1.fSinDelta",
    "Events;2/MHillas_1./MHillas_1.fCosDelta",
    "Events;2/MHillas_2./MHillas_2.fLength",
    "Events;2/MHillas_2./MHillas_2.fWidth",
    "Events;2/MHillas_2./MHillas_2.fDelta",
    "Events;2/MHillas_2./MHillas_2.fSize",
    "Events;2/MHillas_2./MHillas_2.fMeanX",
    "Events;2/MHillas_2./MHillas_2.fMeanY",
    "Events;2/MHillas_2./MHillas_2.fSinDelta",
    "Events;2/MHillas_2./MHillas_2.fCosDelta",
    "Events;2/MStereoPar./MStereoPar.fDirectionX",
    "Events;2/MStereoPar./MStereoPar.fDirectionY",
    "Events;2/MStereoPar./MStereoPar.fDirectionZd",
    "Events;2/MStereoPar./MStereoPar.fDirectionAz",
    "Events;2/MStereoPar./MStereoPar.fDirectionDec",
    "Events;2/MStereoPar./MStereoPar.fDirectionRA",
    "Events;2/MStereoPar./MStereoPar.fTheta2",
    "Events;2/MStereoPar./MStereoPar.fCoreX",
    "Events;2/MStereoPar./MStereoPar.fCoreY",
    "Events;2/MStereoPar./MStereoPar.fM1Impact",
    "Events;2/MStereoPar./MStereoPar.fM2Impact",
    "Events;2/MStereoPar./MStereoPar.fM1ImpactAz",
    "Events;2/MStereoPar./MStereoPar.fM2ImpactAz",
    "Events;2/MStereoPar./MStereoPar.fMaxHeight",
    "Events;2/MStereoPar./MStereoPar.fXMax",
    "Events;2/MStereoPar./MStereoPar.fCherenkovRadius",
    "Events;2/MStereoPar./MStereoPar.fCherenkovDensity",
    # "Events;2/MStereoPar./MStereoPar.fEnergy",
    # "Events;2/MStereoPar./MStereoPar.fEnergyUncertainty",
    # "Events;2/MStereoPar./MStereoPar.fEnergyDiscrepancy",
    "Events;2/MStereoPar./MStereoPar.fPhiBaseLineM1",
    "Events;2/MStereoPar./MStereoPar.fPhiBaseLineM2",
    "Events;2/MStereoPar./MStereoPar.fImageAngle",
    # "Events;2/MStereoPar./MStereoPar.fDispRMS",
    # "Events;2/MStereoPar./MStereoPar.fDispDiff2",
    "Events;2/MStereoPar./MStereoPar.fCosBSangle",
    "Events;2/MPointingPos_2./MPointingPos_2.fZd",
    "Events;2/MPointingPos_2./MPointingPos_2.fAz",
    "Events;2/MHillasTimeFit_1./MHillasTimeFit_1.fP1Grad",
    "Events;2/MHillasTimeFit_2./MHillasTimeFit_2.fP1Grad",
    "Events;2/MMcEvt_1./MMcEvt_1.fImpact",
    "Events;2/MMcEvt_2./MMcEvt_2.fImpact",
    "Events;2/MHillasSrc_1./MHillasSrc_1.fAlpha",
    "Events;2/MHillasSrc_1./MHillasSrc_1.fDist",
    "Events;2/MHillasSrc_1./MHillasSrc_1.fCosDeltaAlpha",
    "Events;2/MHillasSrc_1./MHillasSrc_1.fDCA",
    "Events;2/MHillasSrc_1./MHillasSrc_1.fDCADelta",
    "Events;2/MHillasSrc_2./MHillasSrc_2.fAlpha",
    "Events;2/MHillasSrc_2./MHillasSrc_2.fDist",
    "Events;2/MHillasSrc_2./MHillasSrc_2.fCosDeltaAlpha",
    "Events;2/MHillasSrc_2./MHillasSrc_2.fDCA",
    "Events;2/MHillasSrc_2./MHillasSrc_2.fDCADelta",
]

with uproot.open(superstar_filepath) as f:
    # get the event number
    res["event_number"] = f["Events;2/MMcEvt_1./MMcEvt_1.fEvtNumber"].array(
        library="np"
    )
    res["run_number"] = np.full_like(res["event_number"], run_number)

    for k in params:
        title = k.split("/")[-1]
        res[title] = f[k].array().to_numpy()

    res["images_m1"] = [
        i.tojson() for i in f["Events;2/UprootImageOrig_1"].array(library="np")
    ]
    res["images_m2"] = [
        i.tojson() for i in f["Events;2/UprootImageOrig_2"].array(library="np")
    ]
    res["clean_images_m1"] = [
        i.tojson() for i in f["Events;2/UprootImageOrigClean_1"].array(library="np")
    ]
    res["clean_images_m2"] = [
        i.tojson() for i in f["Events;2/UprootImageOrigClean_2"].array(library="np")
    ]


# also extract images and timing information from the calibrated files
calibrated_res = {
    1: {},
    2: {},
}
for file, telescope in zip(
    [test_run_m1.local_path_mnt, test_run_m2.local_path_mnt], [1, 2]
):
    with uproot.open(file) as f:
        event_numbers = f["Events;1/MMcEvt./MMcEvt.fEvtNumber"].array(library="np")
        calibrated_res[telescope][f"calibrated_event_number_M{telescope}"] = (
            event_numbers.tolist()
        )
        n_events = len(event_numbers)

        # calibrated_res[telescope][f"calibrated_images_M{telescope}"] = (
        #     f["Events;1/MCerPhotEvt./MCerPhotEvt.fPixels/MCerPhotEvt.fPixels.fPhot"]
        #     .array(library="np")
        #     .tolist()
        # )

        arrival_times = []
        timing_data = f["Events;1/MArrivalTime./MArrivalTime.fData"].array()
        for t in timing_data:
            d = t.to_list()
            arrival_times.append(d)

        calibrated_res[telescope][f"calibrated_timing_M{telescope}"] = (
            np.array(arrival_times).reshape(n_events, len(d)).tolist()
        )

        # reformat calibrated res into {event_number: {m1_image: value, m2_image: value, ...}}
        event_key = f"calibrated_event_number_M{telescope}"
        calibrated_res[telescope] = {
            event_number: {k: v[i] for k, v in calibrated_res[telescope].items()}
            for i, event_number in enumerate(calibrated_res[telescope][event_key])
        }

        # remove event numbers over 10000
        calibrated_res[telescope] = {
            k: v for k, v in calibrated_res[telescope].items() if k < 10000
        }


# merge based on event_number
calibrated_merged = pd.DataFrame.from_dict(calibrated_res[1], orient="index").merge(
    pd.DataFrame.from_dict(calibrated_res[2], orient="index"),
    left_index=True,
    right_index=True,
)

# merge with the superstar data
superstar_df = pd.DataFrame.from_dict(res)
merged = superstar_df.merge(
    calibrated_merged, left_on="event_number", right_index=True
)

drop = [
    "MMcEvt_2.fEvtNumber",
    "MMcEvt_1.fEvtNumber",
    "calibrated_event_number_M1",
    "calibrated_event_number_M2",
]

merged = merged.drop(columns=drop)

merged.to_parquet("./test.parquet")
